# 日本国債イールドカーブの主成分分析（シンプル版）

## アプローチ
1. 過去N日分のデータから残存期間の和集合を作成
2. 各日付で3次スプライン補間を使ってイールドカーブを作成
3. 和集合の各残存期間点での金利を補間で算出
4. 主成分分析を実行し、指定成分数で復元
5. 元データと復元データの差分を可視化

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, UnivariateSpline
from sklearn.decomposition import PCA
from datetime import datetime, timedelta
import re
from typing import List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

from data.utils.database_manager import DatabaseManager

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Hiragino Sans', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic']
plt.rcParams['axes.unicode_minus'] = False

## 1. パラメータ設定

In [ ]:
def fetch_bond_data_paginated(db_manager: DatabaseManager, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Supabaseから国債データを取得（ページング対応）
    """
    import requests
    
    url = f'{db_manager.supabase_url}/rest/v1/bond_data'
    all_data = []
    offset = 0
    limit = 1000
    
    print(f"🔄 データ取得中 ({start_date} ~ {end_date})...")
    
    while True:
        query_params = [
            f'select=trade_date,bond_name,due_date,ave_compound_yield',
            f'trade_date=gte.{start_date}',
            f'trade_date=lte.{end_date}',
            f'ave_compound_yield=not.is.null',
            f'due_date=not.is.null',
            f'order=trade_date.asc,due_date.asc',
            f'offset={offset}',
            f'limit={limit}'
        ]
        
        full_url = f"{url}?{'&'.join(query_params)}"
        response = requests.get(full_url, headers=db_manager.headers)
        
        if response.status_code != 200:
            print(f"❌ データ取得失敗: {response.status_code}")
            break
        
        data = response.json()
        if not data:
            break
        
        all_data.extend(data)
        
        if len(data) < limit:
            break
        
        offset += limit
    
    if not all_data:
        return pd.DataFrame()
    
    df = pd.DataFrame(all_data)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df['due_date'] = pd.to_datetime(df['due_date'])
    df['maturity_years'] = (df['due_date'] - df['trade_date']).dt.days / 365.25
    
    # 異常値除去 + 残存期間0.1年以下を除外
    df = df[(df['maturity_years'] > 0.1) & (df['maturity_years'] <= 50)]
    df = df[(df['ave_compound_yield'] > 0) & (df['ave_compound_yield'] < 10)]
    
    print(f"✅ データ取得完了: {len(df)}件")
    print(f"   日付範囲: {df['trade_date'].min().date()} ~ {df['trade_date'].max().date()}")
    print(f"   ユニーク日付数: {df['trade_date'].nunique()}日")
    
    return df


# データベース接続
db_manager = DatabaseManager()

# データ取得範囲を計算（営業日を考慮して余裕を持たせる）
ref_date = pd.to_datetime(REFERENCE_DATE)
start_date = (ref_date - timedelta(days=int(LOOKBACK_DAYS * 2.0))).date()
end_date = ref_date.date()

print(f"分析基準日: {REFERENCE_DATE}")
print(f"データ取得範囲: {start_date} ~ {end_date}")

# データ取得
df_all = fetch_bond_data_paginated(db_manager, str(start_date), str(end_date))

# 分析基準日以前の営業日から過去LOOKBACK_DAYS分を選択
unique_dates = sorted(df_all['trade_date'].unique())
past_dates = [d for d in unique_dates if d <= ref_date]
selected_dates = past_dates[-LOOKBACK_DAYS:]

df = df_all[df_all['trade_date'].isin(selected_dates)].copy()

print(f"\n✅ 分析対象データ:")
print(f"   日付数: {len(selected_dates)}日")
print(f"   日付範囲: {selected_dates[0].date()} ~ {selected_dates[-1].date()}")
print(f"   総レコード数: {len(df)}件")
print(f"   残存期間範囲: {df['maturity_years'].min():.2f}年 ~ {df['maturity_years'].max():.2f}年")

## 2. データ取得

In [ ]:
def fetch_bond_data_paginated(db_manager: DatabaseManager, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Supabaseから国債データを取得（ページング対応）
    """
    import requests
    
    url = f'{db_manager.supabase_url}/rest/v1/bond_data'
    all_data = []
    offset = 0
    limit = 1000
    
    print(f"🔄 データ取得中 ({start_date} ~ {end_date})...")
    
    while True:
        query_params = [
            f'select=trade_date,bond_name,due_date,ave_compound_yield',
            f'trade_date=gte.{start_date}',
            f'trade_date=lte.{end_date}',
            f'ave_compound_yield=not.is.null',
            f'due_date=not.is.null',
            f'order=trade_date.asc,due_date.asc',
            f'offset={offset}',
            f'limit={limit}'
        ]
        
        full_url = f"{url}?{'&'.join(query_params)}"
        response = requests.get(full_url, headers=db_manager.headers)
        
        if response.status_code != 200:
            print(f"❌ データ取得失敗: {response.status_code}")
            break
        
        data = response.json()
        if not data:
            break
        
        all_data.extend(data)
        
        if len(data) < limit:
            break
        
        offset += limit
    
    if not all_data:
        return pd.DataFrame()
    
    df = pd.DataFrame(all_data)
    df['trade_date'] = pd.to_datetime(df['trade_date'])
    df['due_date'] = pd.to_datetime(df['due_date'])
    df['maturity_years'] = (df['due_date'] - df['trade_date']).dt.days / 365.25
    
    # 異常値除去
    df = df[(df['maturity_years'] > 0) & (df['maturity_years'] <= 50)]
    df = df[(df['ave_compound_yield'] > 0) & (df['ave_compound_yield'] < 10)]
    
    print(f"✅ データ取得完了: {len(df)}件")
    print(f"   日付範囲: {df['trade_date'].min().date()} ~ {df['trade_date'].max().date()}")
    print(f"   ユニーク日付数: {df['trade_date'].nunique()}日")
    
    return df


# データベース接続
db_manager = DatabaseManager()

# データ取得範囲を計算（営業日を考慮して余裕を持たせる）
ref_date = pd.to_datetime(REFERENCE_DATE)
start_date = (ref_date - timedelta(days=int(LOOKBACK_DAYS * 2.0))).date()
end_date = ref_date.date()

print(f"分析基準日: {REFERENCE_DATE}")
print(f"データ取得範囲: {start_date} ~ {end_date}")

# データ取得
df_all = fetch_bond_data_paginated(db_manager, str(start_date), str(end_date))

# 分析基準日以前の営業日から過去LOOKBACK_DAYS分を選択
unique_dates = sorted(df_all['trade_date'].unique())
past_dates = [d for d in unique_dates if d <= ref_date]
selected_dates = past_dates[-LOOKBACK_DAYS:]

df = df_all[df_all['trade_date'].isin(selected_dates)].copy()

print(f"\n✅ 分析対象データ:")
print(f"   日付数: {len(selected_dates)}日")
print(f"   日付範囲: {selected_dates[0].date()} ~ {selected_dates[-1].date()}")
print(f"   総レコード数: {len(df)}件")

## 3. カレント債選択とデータ準備

In [ ]:
def extract_bond_maturity(bond_name: str) -> Optional[int]:
    """
    銘柄名から発行年限を抽出
    """
    if bond_name is None:
        return None
    
    patterns = [r'(\d+)年', r'(\d+)Year']
    for pattern in patterns:
        match = re.search(pattern, str(bond_name))
        if match:
            return int(match.group(1))
    return None


def select_current_bonds(daily_df: pd.DataFrame, tolerance: float = 0.01) -> pd.DataFrame:
    """
    同一残存期間（tolerance以内）の銘柄からカレント債（発行年限最小）を選択
    """
    # 発行年限を抽出
    daily_df = daily_df.copy()
    daily_df['bond_maturity'] = daily_df['bond_name'].apply(extract_bond_maturity)
    
    # 残存期間でソート
    daily_df = daily_df.sort_values('maturity_years')
    
    selected_bonds = []
    used_indices = set()
    
    for idx, row in daily_df.iterrows():
        if idx in used_indices:
            continue
        
        # tolerance以内の同一残存期間の銘柄を抽出
        mask = (np.abs(daily_df['maturity_years'] - row['maturity_years']) <= tolerance)
        group = daily_df[mask]
        
        # 発行年限が抽出できた銘柄のみ
        valid_group = group[group['bond_maturity'].notna()]
        
        if len(valid_group) > 0:
            # 発行年限が最小のものを選択（カレント債）
            current_bond = valid_group.loc[valid_group['bond_maturity'].idxmin()]
        else:
            # 発行年限が抽出できない場合は最初の銘柄
            current_bond = group.iloc[0]
        
        selected_bonds.append(current_bond)
        used_indices.update(group.index)
    
    return pd.DataFrame(selected_bonds)


# 残存期間の和集合を作成
all_maturities = df['maturity_years'].unique()
maturity_union = np.sort(all_maturities)

print(f"\n✅ 残存期間の和集合:")
print(f"   次元数: {len(maturity_union)}")
print(f"   範囲: {maturity_union.min():.4f}年 ~ {maturity_union.max():.4f}年")

## 4. スプライン補間でデータ行列を構築

In [ ]:
def interpolate_yield_curve(daily_df: pd.DataFrame, maturity_union: np.ndarray, 
                            tolerance: float = 0.01) -> Optional[np.ndarray]:
    """
    3次スプライン補間でイールドカーブを作成
    """
    # カレント債を選択
    selected = select_current_bonds(daily_df, tolerance=tolerance)
    
    if len(selected) < 4:  # 3次スプラインには最低4点必要
        return None
    
    # ソート
    selected = selected.sort_values('maturity_years')
    
    x = selected['maturity_years'].values
    y = selected['ave_compound_yield'].values
    
    try:
        # 3次スプライン補間
        spline = UnivariateSpline(x, y, k=3, s=0)
        
        # 和集合の各点で評価
        yield_vector = spline(maturity_union)
        
        # 異常値クリップ
        yield_vector = np.clip(yield_vector, 0, 10)
        
        return yield_vector
    except Exception as e:
        # 3次スプラインが失敗した場合は線形補間にフォールバック
        try:
            linear_interp = interp1d(x, y, kind='linear', fill_value='extrapolate')
            yield_vector = linear_interp(maturity_union)
            yield_vector = np.clip(yield_vector, 0, 10)
            return yield_vector
        except:
            return None


# 各日付でイールドカーブをベクトル化
print("\n🔄 各日付のイールドカーブを補間中...")

dates = []
yield_vectors = []
failed_dates = []

for trade_date, daily_df in df.groupby('trade_date'):
    yield_vector = interpolate_yield_curve(daily_df, maturity_union, tolerance=SAME_MATURITY_TOLERANCE)
    
    if yield_vector is not None:
        dates.append(trade_date.strftime('%Y-%m-%d'))
        yield_vectors.append(yield_vector)
    else:
        failed_dates.append(trade_date.strftime('%Y-%m-%d'))

yield_matrix = np.array(yield_vectors)

print(f"\n✅ データ行列構築完了")
print(f"   形状: {yield_matrix.shape} (日数 × 残存期間点数)")
print(f"   日付範囲: {dates[0]} ~ {dates[-1]}")
print(f"   成功: {len(dates)}日, 失敗: {len(failed_dates)}日")

if failed_dates:
    print(f"\n⚠️ 補間失敗した日付（データ点が4点未満）:")
    for d in failed_dates:
        print(f"     {d}")

## 5. 主成分分析

In [ ]:
# PCA実行
pca = PCA(n_components=N_COMPONENTS)
scores = pca.fit_transform(yield_matrix)

print(f"\n✅ PCA実行完了")
print(f"   主成分数: {N_COMPONENTS}")
print(f"   累積寄与率:")
cumsum = np.cumsum(pca.explained_variance_ratio_)
for i in range(min(5, N_COMPONENTS)):
    print(f"     PC{i+1}: {cumsum[i]*100:.2f}%")

## 6. 主成分ベクトルの可視化

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

for i in range(3):
    ax.plot(maturity_union, pca.components_[i], 'o-', 
            label=f'第{i+1}主成分 ({pca.explained_variance_ratio_[i]*100:.1f}%)', 
            alpha=0.7, markersize=2)

ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('残存期間（年）', fontsize=12)
ax.set_ylabel('主成分ベクトル', fontsize=12)
ax.set_title('主成分ベクトル（固有ベクトル）', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. 復元と誤差可視化

In [ ]:
# 指定成分数で復元
scores_partial = scores[:, :N_COMPONENTS_USE]
components_partial = pca.components_[:N_COMPONENTS_USE, :]
reconstructed_yields = scores_partial @ components_partial + pca.mean_

print(f"\n✅ 利回り復元完了（第{N_COMPONENTS_USE}主成分まで使用）")

# 可視化する日付のインデックスを取得
plot_indices = [dates.index(d) for d in PLOT_DATES if d in dates]

if len(plot_indices) == 0:
    print(f"\n⚠️ 指定された日付がデータに存在しません")
    print(f"   利用可能な最新日付: {dates[-1]}")
else:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # 上段: 元データと復元データ
    ax1 = axes[0]
    for idx in plot_indices:
        date_str = dates[idx]
        ax1.plot(maturity_union, yield_matrix[idx], 'o-', 
                label=f'{date_str} (元データ)', alpha=0.7, markersize=2)
        ax1.plot(maturity_union, reconstructed_yields[idx], 's--', 
                label=f'{date_str} (復元)', alpha=0.7, markersize=2)
    
    ax1.set_xlabel('残存期間（年）', fontsize=12)
    ax1.set_ylabel('利回り（%）', fontsize=12)
    ax1.set_title(f'イールドカーブ: 元データ vs 復元（第{N_COMPONENTS_USE}主成分）', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # 下段: 復元誤差
    ax2 = axes[1]
    for idx in plot_indices:
        date_str = dates[idx]
        error = (yield_matrix[idx] - reconstructed_yields[idx]) * 100  # bp変換
        ax2.plot(maturity_union, error, 'o-', label=f'{date_str}', alpha=0.7, markersize=2)
    
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax2.set_xlabel('残存期間（年）', fontsize=12)
    ax2.set_ylabel('復元誤差（bp）', fontsize=12)
    ax2.set_title(f'復元誤差（元データ - 復元データ）', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 8. 誤差統計

In [ ]:
errors_bp = (yield_matrix - reconstructed_yields) * 100

print("\n" + "="*60)
print(f"復元誤差統計（第{N_COMPONENTS_USE}主成分まで使用）")
print("="*60)
print(f"全期間:")
print(f"  平均絶対誤差: {np.abs(errors_bp).mean():.2f} bp")
print(f"  最大絶対誤差: {np.abs(errors_bp).max():.2f} bp")
print(f"  標準偏差: {errors_bp.std():.2f} bp")
print("="*60)

for plot_date in PLOT_DATES:
    if plot_date in dates:
        idx = dates.index(plot_date)
        date_errors = errors_bp[idx]
        print(f"\n{plot_date}:")
        print(f"  平均絶対誤差: {np.abs(date_errors).mean():.2f} bp")
        print(f"  最大絶対誤差: {np.abs(date_errors).max():.2f} bp")
        print(f"  標準偏差: {date_errors.std():.2f} bp")